In [10]:
import numpy as np
from pathlib import Path
from quick_calib.camera import get_calib_from_vp, get_vp_from_calib

# vanishing points for each video
# labeled
vanishing_points = np.array([
    # labeled
    [614, 410],
    [591, 382],
    [639, 408],
    [595, 422],
    [631, 424],
    # unlabed
    [622, 427],
    [562, 403], # 
    [542, 420], # 
    # [632, 542], # 
    [632, 453], # 
    [600, 435],
])

OUTPUT_DIR = Path("../data/calib_challenge/test/")
OUTPUT_DIR.mkdir(exist_ok=True)

for i in range(5):
    _, pitch, yaw = get_calib_from_vp(vanishing_points[i])

    # get nb frames in video
    nb_frames = len(list(Path(f"../data/calib_challenge/labeled/{i}/").glob("*.jpg")))

    # make numpy array of shape (nb_frames, 2)
    # where each row is (pitch, yaw)
    calib = np.array([[pitch, yaw]] * nb_frames)
    np.savetxt(OUTPUT_DIR / f"{i}.txt", calib)

for i in range(5, 10):
    _, pitch, yaw = get_calib_from_vp(vanishing_points[i])
    nb_frames = len(list(Path(f"../data/calib_challenge/unlabeled/{i}/").glob("*.jpg")))
    calib = np.array([[pitch, yaw]] * nb_frames)
    np.savetxt(OUTPUT_DIR / f"{i}.txt", calib)

In [7]:
# viz calib
import cv2
from tqdm import trange

for i in range(5, 10):
    video = cv2.VideoCapture(f"../data/calib_challenge/unlabeled/{i}.hevc")
    output = f"../data/calib_challenge/{i}.mp4"
    
    labels = np.loadtxt(f"../data/calib_challenge/test/{i}.txt")

    # get video size
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))
    
    size = (frame_width, frame_height)
    result = cv2.VideoWriter(output, 
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            20, size)


    # overlay the yaw data on the video
    for frame_idx in trange(labels.shape[0]):
        # get the next frame
        ret, frame = video.read()
        if not ret:
            break

        # overlay on the frame
        pitch, yaw = labels[frame_idx]
        vanishing_point = get_vp_from_calib(pitch, yaw)
        cv2.circle(frame, (int(vanishing_point[0]), int(vanishing_point[1])), 5, (0, 0, 255), -1)

        result.write(frame)

    video.release()
    result.release()

100%|██████████| 1200/1200 [00:08<00:00, 142.53it/s]
